In [3]:
import pandas as pd
import json
import requests
import time
from tqdm import tqdm

import mariadb
from sqlalchemy import create_engine

def make_binary(x):
    if x!=0:
        return 1
    return 0


In [33]:
def make_jongna_data(query,sortStartDate,sortEndDate):
    url = f'https://search-api.joongna.com/v25/search/product'
    param = {
        "filter":{
            "dateFilterParameter":{
                "sortEndDate":f"{sortEndDate}",
                "sortStartDate":f"{sortStartDate}"
            },
            "flawedYn":0,
            "fullPackageYn":0,
            "limitedEditionYn":0,
    #         "maxPrice":2000000000,
    #         "minPrice":2,
            "tradeType":0
        },
        "productFilter":"ALL",
        "productStates":[
            0,
            1,
            99
        ],
        "searchQuantity":10000,
        "osType":2,
        "searchWord":f'{query}',
        "sort":"RECENT_SORT",
    }
    response= requests.post(url,json=(param))
    data=response.json()['data']
    col=['title'] # 0 1 3]
    # 데이터프레임 화 , 중복 제거
    df=pd.DataFrame(data['items']).drop_duplicates('title')[col]
    # nan 제거
    df=df.dropna()
    # is_sold 처리
    # 매입 게시물 제거
    buy = df['title'].str.contains('삽니|매입|사요', na=False)
    df['label']=0
    df.loc[buy,'label']=1
    print(sortStartDate,sortEndDate,len(df))
    return df

In [52]:
query_list=['문화상품권','아이폰','맥북 프로','그램','갤럭시','해피머니','상품권','에어팟','아이패드']
final= make_jongna_data(query[0],"2022-05-14" ,"2022-05-20")
day_list=[(5,13),(5,7),(4,30)
          ,(4,23),(4,16),(4,9),(3,31),(3,24),(3,17),(3,10),(2,28),(2,21)
,(2,14),(2,7),(1,31),(1,24),(1,17),(1,10)]
for query in query_list[1:]:
    for mon,end_d in (day_list):
        day=end_d-6
        if day<10:
            day=f"0{day}"
        if end_d <10:
            end_d=f"0{end_d}"
        final=pd.concat([final,make_jongna_data(query,f"2022-0{mon}-{day}" ,f"2022-0{mon}-{end_d}")])

2022-05-14 2022-05-20 165
2022-05-07 2022-05-13 2074
2022-05-01 2022-05-07 1789
2022-04-24 2022-04-30 1961
2022-04-17 2022-04-23 2051
2022-04-10 2022-04-16 1984
2022-04-03 2022-04-09 1926
2022-03-25 2022-03-31 2352
2022-03-18 2022-03-24 1885
2022-03-11 2022-03-17 1639
2022-03-04 2022-03-10 1519
2022-02-22 2022-02-28 1448
2022-02-15 2022-02-21 1351
2022-02-08 2022-02-14 1228
2022-02-01 2022-02-07 1432
2022-01-25 2022-01-31 1348
2022-01-18 2022-01-24 1670
2022-01-11 2022-01-17 1465
2022-01-04 2022-01-10 1796
2022-05-07 2022-05-13 882
2022-05-01 2022-05-07 863
2022-04-24 2022-04-30 781
2022-04-17 2022-04-23 839
2022-04-10 2022-04-16 773
2022-04-03 2022-04-09 794
2022-03-25 2022-03-31 798
2022-03-18 2022-03-24 745
2022-03-11 2022-03-17 738
2022-03-04 2022-03-10 703
2022-02-22 2022-02-28 658
2022-02-15 2022-02-21 738
2022-02-08 2022-02-14 703
2022-02-01 2022-02-07 651
2022-01-25 2022-01-31 632
2022-01-18 2022-01-24 708
2022-01-11 2022-01-17 719
2022-01-04 2022-01-10 760
2022-05-07 2022-05-1

ChunkedEncodingError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))

In [53]:
final

,title,label
0,갤럭치 워치4 40mm 블랙 미개봉 18만원에 팝니다,0
1,일본 나비 동경 진품 명품 골동 고미술품 엔틱 앤틱 갤,0
2,갤S노트9/최상품A/정품판매AS가능[부천]314번[송내역/중고폰],0
3,삼성 갤러시 노트10플러스 512기가 직거래.퀵.택배거래 가능!!,0
4,갤러시 버즈 프로 ( 미개봉세제품 ) 부산서면,0
...,...,...
1944,에어팟 가죽 케이스,0
1946,에어팟 프로 미개봉 삽니다 대구 경북대,1
1948,에어팟프로3세대,0
1951,에어팟 3 판매합니다,0


In [58]:
spam=final[final['label']==1].copy()
yes=final[final['label']==0].copy()
spam

,title,label
635,갤즈 만화책 전권 삽니다,1
0,<매입>아이폰13 미니 프로 맥스 전기종매입<중고폰/자급제/개통폰>,1
2,[24시어디든지방문매입]아이폰 갤럭시 전기종켜지면다사요구매LCD파손액파자급제,1
3,"[24시어디든매입]갤럭시,아이폰모두구매LCD파손액파박스폰자급제미개봉",1
5,"아이폰13 PRO MAX 프로 맥스 자급제미개봉,당일개통,박스폰삽니다",1
...,...,...
1215,에어팟2 유선 미개봉 삽니다,1
1232,에어팟맥스 미개봉 삽니다. (40),1
1427,에어팟 프로 삽니다,1
1688,에어팟3 미개봉 안전 18에 삽니다,1


In [59]:
yes.sample(frac=0.2)

,title,label
2777,크리스피 모바일상품권 8000원,0
3218,"아이폰XsMax 실리콘케이스 레드(새상품, 정품)",0
2247,gs25 모바일상품권 만원권팝니다,0
9444,아이폰 7 plus 플러스 박스 팔아요.,0
3323,신세계 상품권 모바일 교환권 25만원,0
...,...,...
1785,31417 갤럭시S10 프리즘화이트 128GB 23만원에 판매합니다,0
116,아이폰12미니 블랙 64GB 45만 07353 수원,0
7922,(196018)갤럭시S21 바이올렛 256기가 A급 44만원,0
7467,갤럭시 S10 라이트 젤리케이스,0


In [79]:
import numpy as np
from sklearn.model_selection import train_test_split
total=pd.concat([yes.sample(frac=0.2).copy(),spam])
# total.iloc[:5000,1]=1
total.to_csv("total.csv",index=False,encoding='utf-8-sig')

In [77]:
# x_train, x_test, y_train, y_test = train_test_split(total['title'], total['label'], test_size=0.3, random_state=777, stratify= total['label']) 
# x_train.to_csv("x_train.csv",index=False,encoding='utf-8-sig')
# x_test.to_csv("x_test.csv",index=False,encoding='utf-8-sig')
# y_train.to_csv("y_train.csv",index=False,encoding='utf-8-sig')
# y_test.to_csv("y_test.csv",index=False,encoding='utf-8-sig')

In [78]:
total.iloc[0:10000,1]

2061    1
8053    1
6764    1
8173    1
902     1
       ..
689     0
108     0
1628    0
4178    0
874     0
Name: label, Length: 10000, dtype: int64